In [ ]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2023-03-16 17:23:59--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.03s   

2023-03-16 17:24:00 (37.0 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel
block_size = 32 # The maximum context length for predictions
max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
num_eval_batches = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0
torch.manual_seed(1337)

In [ ]:
with open('input.txt', 'r', encoding = 'utf-8') as f:
  text = f.read()

In [ ]:
character_length = len(text)
print(character_length)

1115394


In [ ]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [ ]:

char_to_int = {char: i for i, char in enumerate(chars)}
int_to_char = {i: char for i, char in enumerate(chars)}

encode = lambda s: [char_to_int[char] for char in s]
decode = lambda l: ''.join([int_to_char[i] for i in l])

In [ ]:
data = torch.tensor(encode(text), dtype = torch.long)
print(data.shape, data.dtype)
print(data[:1000])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [ ]:
n = int(0.9 * len(data)) # first 80% of the text will be used to train
train_data = data[:n]
val_data = data[n:]

In [ ]:
# data loading
def generate_small_batch(split):
    # generate a small batch of data with input sequences x and target sequences y
    dataset = train_data if split == 'train' else val_data
    start_indices = torch.randint(len(dataset) - block_size, (batch_size,))
    input_sequences = torch.stack([dataset[i:i+block_size] for i in start_indices])
    target_sequences = torch.stack([dataset[i+1:i+block_size+1] for i in start_indices])
    input_sequences, target_sequences = input_sequences.to(device), target_sequences.to(device)
    return input_sequences, target_sequences


In [ ]:
@torch.no_grad()
def estimate_loss():
    # estimate the loss for the training and validation sets
    loss_dict = {}
    model.eval()

    for dataset in ['train', 'val']:
        batch_losses = torch.zeros(num_eval_batches)
        for batch_idx in range(num_eval_batches):
            input_seqs, target_seqs = generate_small_batch(dataset)
            logits, loss = model(input_seqs, target_seqs)
            batch_losses[batch_idx] = loss.item()
        mean_loss = batch_losses.mean()
        loss_dict[dataset] = mean_loss

    model.train()
    return loss_dict


In [ ]:
class SelfAttentionHead(nn.Module):
    """One head of self-attention."""

    def __init__(self, head_size):
        super().__init__()
        self.key_projection = nn.Linear(n_embd, head_size, bias=False)
        self.query_projection = nn.Linear(n_embd, head_size, bias=False)
        self.value_projection = nn.Linear(n_embd, head_size, bias=False)
        self.lower_triangular_mask = torch.tril(torch.ones(block_size, block_size)).detach()
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        batch_size, seq_length, emb_size = x.shape

        # project x to key, query and value vectors
        keys = self.key_projection(x)   # (batch_size, seq_length, head_size)
        queries = self.query_projection(x) # (batch_size, seq_length, head_size)
        values = self.value_projection(x) # (batch_size, seq_length, head_size)

        # compute attention scores ("affinities")
        affinities = torch.bmm(queries, keys.transpose(1, 2)) * (emb_size ** -0.5) # (batch_size, seq_length, seq_length)
        affinities = affinities.masked_fill(self.lower_triangular_mask[:seq_length, :seq_length] == 0, float('-inf')) # (batch_size, seq_length, seq_length)
        affinities = F.softmax(affinities, dim=-1) # (batch_size, seq_length, seq_length)
        affinities = self.dropout(affinities)

        # perform the weighted aggregation of the values
        weighted_values = torch.bmm(affinities, values) # (batch_size, seq_length, head_size)
        return weighted_values


In [ ]:

class MultiHeadAttention(nn.Module):
    """ Multiple heads of self-attention in parallel. """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([SelfAttentionHead(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(num_heads * head_size, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([head(x) for head in self.heads], dim=-1)
        out = self.proj(out)
        out = self.dropout(out)
        return out


In [ ]:
class FeedForward(nn.Module):
    """ A simple linear layer followed by a non-linearity and dropout. """

    def __init__(self, n_embd):
        super().__init__()

        # Define the feedforward network as a sequential module of linear layers, 
        # ReLU activation and dropout layer.
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd), # Input to hidden layer
            nn.ReLU(), # Non-linearity
            nn.Linear(4 * n_embd, n_embd), # Hidden to output layer
            nn.Dropout(dropout), # Dropout to reduce overfitting
        )

    def forward(self, x):
        # Pass input tensor through the feedforward network
        out = self.net(x)
        return out


In [ ]:
class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        """
        Args:
        - n_embd (int): embedding dimension
        - n_head (int): number of heads
        """
        super().__init__()
        
        # create the self-attention layer with multiple heads
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        
        # create the feedforward network
        self.ffwd = FeedFoward(n_embd)
        
        # create the layer normalization layers
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        """
        Args:
        - x (torch.Tensor): input tensor of shape (batch_size, sequence_length, embedding_dim)
        
        Returns:
        - output tensor of the same shape as the input
        """
        # apply the self-attention layer with layer normalization and residual connection
        x = x + self.sa(self.ln1(x))
        
        # apply the feedforward network with layer normalization and residual connection
        x = x + self.ffwd(self.ln2(x))
        
        # return the output
        return x



In [ ]:
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # Define token embedding table and position embedding table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)

        # Define transformer blocks
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])

        # Define final layer norm and linear projection layer
        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        # idx and targets are both (B,T) tensor of integers
        B, T = idx.shape

        # Apply token and position embeddings
        tok_emb = self.token_embedding_table(idx)  # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device))  # (T,C)
        x = tok_emb + pos_emb  # (B,T,C)

        # Apply transformer blocks
        x = self.blocks(x)  # (B,T,C)

        # Apply final layer norm and linear projection
        x = self.ln_f(x)  # (B,T,C)
        logits = self.lm_head(x)  # (B,T,vocab_size)

        if targets is None:
            # If no target is given, return None for the loss
            loss = None
        else:
            # Flatten logits and targets for computing the loss
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)

            # Compute the cross-entropy loss between the logits and targets
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # Generate new tokens given a sequence of input indices (idx)
        # idx is (B, T) array of indices in the current context

        for _ in range(max_new_tokens):
            # Crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]

            # Get the predictions
            logits, loss = self(idx_cond)

            # Focus only on the last time step
            logits = logits[:, -1, :]  # becomes (B, C)

            # Apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1)  # (B, C)

            # Sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1)  # (B, 1)

            # Append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1)  # (B, T+1)

        return idx


In [ ]:
model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

0.209729 M parameters


In [ ]:
optimizer = torch.optim.AdamW(m.parameters(), lr = learning_rate)

In [ ]:
for i in range(max_iters):
    # Evaluate loss on train and val sets
    if i % eval_interval == 0 or i == max_iters - 1:
        losses = estimate_loss()
        print(f"step {i}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # Sample a batch of data
    x_batch, y_batch = generate_small_batch('train')

    # Compute loss
    logits, loss = model(x_batch, y_batch)

    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

step 0: train loss 1.6583, val loss 1.8090
step 100: train loss 1.6489, val loss 1.8114
step 200: train loss 1.6500, val loss 1.8176
step 300: train loss 1.6496, val loss 1.8112
step 400: train loss 1.6433, val loss 1.8118
step 500: train loss 1.6489, val loss 1.8199
step 600: train loss 1.6447, val loss 1.8088
step 700: train loss 1.6301, val loss 1.7941
step 800: train loss 1.6168, val loss 1.8036
step 900: train loss 1.6273, val loss 1.7913
step 1000: train loss 1.6200, val loss 1.7939
step 1100: train loss 1.6155, val loss 1.7885
step 1200: train loss 1.6072, val loss 1.7800
step 1300: train loss 1.6056, val loss 1.7844
step 1400: train loss 1.6059, val loss 1.7953
step 1500: train loss 1.6079, val loss 1.7728
step 1600: train loss 1.6032, val loss 1.7659
step 1700: train loss 1.5982, val loss 1.7873
step 1800: train loss 1.5874, val loss 1.7795
step 1900: train loss 1.6027, val loss 1.7722
step 2000: train loss 1.5951, val loss 1.7832
step 2100: train loss 1.5903, val loss 1.7696


In [ ]:
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))


sO&&d$TJb3ShxWcM&TqNqJ3-WBDE.f
ocNhhSr phtwGUw?h3EdVv:E $gQpKT:-ciAMUfhN&D'MEHpbzmwJC:
D-vU3-KE3OW;g:bfKcpizyVZT:g??mXzg?orAyGeJOCtos?c;GRWiKeqUzJMzN3wF-OIzAQUBhXrXeAmeqhpcJUw.?Mo&pVTKeHoZhaHDsjtUpI?UvwrqXKjcfFzNYwUptPNzpi'N.aUJhPubn,c3fO;3!o OnA-bF?FwOAcbszn&PcpcusQD!A'hXeWko$oQcoc&HUBc$ nU,JPSDPfjdO:utqU,YEVnooRB ;'z$m!JVqhOxfohe?w.XTE.hphitJkleKV'Uk!Slb:.IpKEVgvvSNPmtAPOqmfYvGeJAcbOEXojvjUgJ'$A&EZcNbc-yXMhAQyzJKIgvB,xfqBJBh3cOOcF?lwaSXbkgzee?B
eiO&zBwNcph&UiowU coDh&foEXwHblrio&ugpKTjDH!NPpjLGbmOWsj,NvA-VjrJhpC'Ntgq!zEqfk-&J?nqNMStevF3Ey;PXXHWHnc$TFUf$QcNU&,JeqKp!
y,zbzdNE-zqt$&&U!h:oH.!kkvJw.wEGuE CtKEOJr3W
D$iGKEUfpw&lzcLUFoJLFBVcx!,LONJJw,bMaNfrablWdqsKcH,XRUplAehiMTYboAKd-!cDDLCFA:FnJE!okfpL,pZD-n!;ITT'C$kobtsHNzWheJcVclD&$3pWUq3,T'-F'OamMcpCVjKtNAQLSFN cEfLPe-roVoNX?Dd$qcBQ,DvnD'E!gKcpTWAau.!'Euvyo3D'saOyol:ueX,ppZRZqvLovQlHM$ec :YztKpYeMROQOLbuASKcp?fjS?mYnzC&,GCmMNeRKNJh;:AYXwiSfqc.Vh,fEy?,?OpZgw-b?hUwtJuIPNCO.pqyV;V?mvKNWjcA'jO'LfoqPef!DNyccAsDK?LakXJso$F.FPFgwTtW33bR$YdoHc

In [ ]:
x[:,0].mean(), x[:,0].std()
x[0,:].mean(), x[0,:].std()

(tensor(0.0496), tensor(0.7829))